In [1]:
import os

In [2]:
%pwd

'/Users/rahatkapoor/Desktop/ML Projects/Text summarizer/Text-summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rahatkapoor/Desktop/ML Projects/Text summarizer/Text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *

In [8]:
import sys
from pathlib import Path

# Assuming your notebook is located in 'Text-summarizer/research'
current_dir = Path.cwd()  # Get the current working directory of the notebook
project_dir = current_dir.parent.parent  # Go up two levels to reach 'Text-summarizer'

# Add 'src' directory to sys.path
src_dir = project_dir / 'src'
sys.path.insert(0, str(src_dir))

# Now you can import from your package
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


AttributeError: 'TestCase' object has no attribute 'assertRaisesRegexp'

In [9]:
!pip install --upgrade ensure


  Attempting uninstall: ensure
    Found existing installation: ensure 1.0.2
    Uninstalling ensure-1.0.2:
      Successfully uninstalled ensure-1.0.2


In [10]:
import sys
from pathlib import Path

# Assuming your notebook is located in 'Text-summarizer/research'
current_dir = Path.cwd()  # Get the current working directory of the notebook
project_dir = current_dir.parent.parent  # Go up two levels to reach 'Text-summarizer'

# Add 'src' directory to sys.path
src_dir = project_dir / 'src'
sys.path.insert(0, str(src_dir))

# Now you can import from your package
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e